<center>

# **"Rainbow" 3D animation**

Small code to be able to quiclky visualise the 3D data gotten for the Rainbow event. To run the code, there is a .venv/ python environment (and the equivalent requirements.txt file) in the same folder that has the necessary modules.
</center>

<font color='orange'> NOTE: </font> If on the remote server you are not using an IDE that has a jupyter notebook visualisation tool integrated inside it, then you'll need to connect through an ssh tunnel:

- connect to one of the remote servers through ssh 
- create a Jupyter Lab instance in --no-bowser mode and specify the port, i.e. type **jupyter-lab --no-browser --port=8080** 
- in local, open the ssh tunnel, i.e. type **ssh -NL 8080:localhost:8080 {username}@{hostname}** 
- paste the link gotten from the Jupyter Lab instance in your browser

In [1]:
# Necessary imports
import os
import re
import k3d
import threading
import numpy as np
import ipywidgets as widgets
from astropy.io import fits
from scipy.io import readsav
from IPython.display import display

In [2]:
# Important paths
main_path = '/home/avoyeux/old_project/avoyeux/'
# cubes_path = os.path.join(main_path, 'cubes')
cubes_path = os.path.join(main_path, 'Cubes')
texture_path = os.path.join(main_path, 'Textures')

### **Preparing the data**

In this part, the data cubes are just uploaded to the notebook. Furthermore, they are seperated in 4 different categories for easier visualisation (i.e. the seperations between the different duplicates).

In [3]:
# Setting the cube name pattern (only cube{:03d}.save files are kept)
pattern = re.compile(r'cube\d{3}\.save')

# Getting the cube names and sorting it so that they are in the right order
cube_names = [cube_name for cube_name in os.listdir(cubes_path) if pattern.match(cube_name)]
cube_names.sort()

print('Visual check of the cube names:')
print(cube_names)

Visual check of the cube names:
['cube085.save', 'cube086.save', 'cube087.save', 'cube088.save', 'cube089.save', 'cube090.save', 'cube091.save', 'cube092.save', 'cube093.save', 'cube094.save', 'cube095.save', 'cube096.save', 'cube097.save', 'cube098.save', 'cube099.save', 'cube100.save', 'cube101.save', 'cube102.save', 'cube103.save', 'cube104.save', 'cube105.save', 'cube215.save', 'cube216.save', 'cube217.save', 'cube218.save', 'cube219.save', 'cube220.save', 'cube221.save', 'cube222.save', 'cube223.save', 'cube224.save', 'cube225.save', 'cube226.save', 'cube227.save', 'cube228.save', 'cube229.save', 'cube230.save', 'cube231.save', 'cube232.save', 'cube233.save', 'cube234.save', 'cube235.save', 'cube236.save', 'cube237.save', 'cube238.save', 'cube239.save', 'cube240.save', 'cube241.save', 'cube242.save', 'cube243.save', 'cube244.save', 'cube245.save', 'cube345.save', 'cube346.save', 'cube347.save', 'cube348.save', 'cube349.save', 'cube350.save', 'cube351.save', 'cube352.save', 'cube35

In [32]:
# Importing the necessary data
cubes = [readsav(os.path.join(cubes_path, cube_name)).cube for cube_name in cube_names]  
cubes1 = [readsav(os.path.join(cubes_path, cube_name)).cube1 for cube_name in cube_names]  
cubes2 = [readsav(os.path.join(cubes_path, cube_name)).cube2 for cube_name in cube_names]  
cubes = np.stack(cubes, axis=0) # all data
cubes1 = np.stack(cubes1, axis=0) # line of sight seen from STEREO 
cubes2 = np.stack(cubes2, axis=0) # line of sight seen from SDO

# Seperating the data
cubes0 = cubes != 0  # all the data
filters1 = (cubes == 3) & (cubes==7) # no duplicates seen from SDO
filters2 = (cubes == 5) & (cubes==7) # no duplicates seen from STEREO
filters3 = (cubes == 7)  # no  duplicates

cubes01 = np.zeros(np.shape(cubes))
cubes02 = np.zeros(np.shape(cubes))
cubes03 = np.zeros(np.shape(cubes)) 

cubes01[filters1] = 1
cubes02[filters2] = 1
cubes03[filters3] = 1

In [45]:
# Other useful data for the plots
trace_cubes = np.any(cubes, axis=0) # basically shows the "trace" of all the data
trace_cubes03 = np.any(cubes03, axis=0) # shows the "trace" of the no duplicates data

<font color='red'> **WEIRD:** </font>
The next code is just to check that I have uploaded the right data set (I initially made some mistakes). That being said, I have no clue how some of the values are equal to 2...

In [ ]:
print(f'np.unique(cubes) gives {np.unique(cubes)}')

np.unique(cubes) gives [0. 1. 3. 5. 7.]


### **Where is the Sun?**

This small bit of code is just to upload the necessary data and find the Sun's radius + center position in the cubes reference frame. 

In [33]:
# Reference data 
first_cube_name = os.path.join(cubes_path, 'cube085.save')

# Initial data values
solar_r = 6.96e5 
length_dx = readsav(first_cube_name).dx
length_dy = readsav(first_cube_name).dy
length_dz = readsav(first_cube_name).dz
x_min = readsav(first_cube_name).xt_min
y_min = readsav(first_cube_name).yt_min
z_min = readsav(first_cube_name).zt_min

# The Sun's radius
radius_index = solar_r / length_dx  # TODO: need to change this if dx!=dy!=dz.

# The Sun center's position
x_index = abs(x_min) / length_dx 
y_index = abs(y_min) / length_dy 
z_index = abs(z_min) / length_dz 
sun_center = [x_index, y_index, z_index]

#### **Creating the Sun**

To create a sphere, the most efficient method is just to create a point and give it the radius of the Sun. That being said, I wasn't able to add a texture when using k3d.points(). I also created a spherical mesh but when I tried, I wasn't able to make use of the k3d.mesh.texture argument (in conjuction with k3d.mesh.texture_file_type). In my honest opinion, I don't thing it works, at least not as explained in the API. Was also unsuccesful when using the k3d.surface() and k3d.volume() options. \
Hence, in the following code I am using the worst possible method, i.e. creating a spherical cloud of points for which I give color values that map out an image of the Sun. 

##### AIA 30.4nm synoptics map

This initial part is just to import the texture (i.e. the synoptics map) and do the usual manipulations to have "nice visuals" (i.e. cutting the high and low values and changing it to a logarithmic scale).

In [34]:
# Importing AIA 33.5nm synoptics map
hdul = fits.open(os.path.join(texture_path, 'syn_AIA_304_2012-07-23T00-00-00_a_V1.fits'))
image = hdul[0].data  # (960, 1920) monochromatic image

# Image shape
height, width = image.shape

# Image treatment
lower_cut = np.nanpercentile(image, 0.5)
upper_cut = np.nanpercentile(image, 99.99)
image[image < lower_cut] = lower_cut
image[image > upper_cut] = upper_cut

# Replacing nan values to the lower_cut 
nw_image = np.where(np.isnan(image), lower_cut, image)  # TODO: would need to change the nan values to the interpolation for the pole

# Changing values to a logarithmic scale
nw_image = np.log(nw_image)

# Creating a 1D array of the color values
def Colours_1D(length_x, length_y):
    x_indices = length_x[:, np.newaxis]
    y_indices = length_y[np.newaxis, :]
    colouring = nw_image[x_indices, y_indices]
    return colouring.flatten()

def Colours_to_hex(colours):
    normalized_colours = (colours - np.min(colours)) / (np.max(colours) - np.min(colours))
    blue_val = (normalized_colours * 255).astype('int')
    return (blue_val << 16) + (blue_val << 8) + blue_val

In [35]:
####### SPHERICAL CLOUD OF POINTS #######

# Initialisation
N = int(960) # number of points in the theta direction
theta = np.linspace(0, np.pi, N)  # latitude of the points
phi = np.linspace(0, 2 * np.pi, 2 * N)  # longitude of the points
theta, phi = np.meshgrid(theta, phi)  # the subsequent meshgrid

# Convertion to cartesian coordinates
x = radius_index * np.sin(theta) * np.cos(phi) + sun_center[0]
y = radius_index * np.sin(theta) * np.sin(phi) + sun_center[1]
z = radius_index * np.cos(theta) + sun_center[2] 

# Creation of the position of the spherical cloud of points
points = np.array([x, y, z]).T 

# The corresponding image indexes to get the colors
image_x = np.linspace(0, height - 1, points.shape[0]).astype('int')
image_y = np.linspace(0, width - 1, points.shape[1]).astype('int')

In [36]:
####### HEX COLOR LIST #######

# Creating the RGB color list
colours = Colours_1D(image_x, image_y)

# Creating the subsequent Hex color list
hex_colors = Colours_to_hex(colours)

In [17]:
# ####### IMAGE AND COLORING STUFF #######

# # Opening the texture image
# image = imread(os.path.join(texture_path, 'texturejpeg.jpg'))  # in RGB format

# # Image shape
# height, width, _ = image.shape

# # Creating an array of the RGB values (with the same size than the number of points)
# def RGB_1D_colors(length_x, length_y):
#     x_indices = length_x[:, np.newaxis] 
#     y_indices = length_y[np.newaxis, :]
#     coloring = image[x_indices, y_indices]  # == doing a double for loop. Just learnt about this method.
#     return np.concatenate(coloring, axis=0, dtype='int')

# # Function to change RGB values to Hex color values as integers
# def rgb_to_hex(rgb):
#     """
#     The output is not a Hex color string (as it normally should be) but integers as it's the only accepted color value input...
#     """
#     return (rgb[:, 0] << 16) + (rgb[:, 1] << 8) + rgb[:, 2]

In [18]:
# ####### HEX COLOR LIST #######

# # Creating the RGB color list
# rgb_colors = RGB_1D_colors(image_x, image_y)

# # Creating the subsequent Hex color list
# hex_colors = rgb_to_hex(rgb_colors)

##### **Adding stars in the background**

Just adding some stars for a nicer visual.

In [37]:
# Stars spherical positions 
stars_N = 400  # total number of stars
stars_radius = np.random.uniform(radius_index * 15, radius_index * 30, stars_N)
stars_theta = np.random.uniform(0, np.pi, stars_N)
stars_phi = np.random.uniform(0, 2 * np.pi, stars_N)

# To cartesian
stars_x = stars_radius * np.sin(stars_theta) * np.cos(stars_phi) + sun_center[0]
stars_y = stars_radius * np.sin(stars_theta) * np.sin(stars_phi) + sun_center[1]
stars_z = stars_radius * np.cos(stars_theta) + sun_center[2]

# Cartesian positions
stars_points = np.array([stars_x, stars_y, stars_z]).T

<center>

## **MAIN PART**
</center>

Main bulk of the code where the visualisation is done, primarily using the k3d library. There is a play/pause button for the animation. Also, for now the sleep() timer between each image is set to 2 seconds to make sure no lag occures. The compression_level is also set to 5 (goes from -1 to 9), but you can probably go lower. I just know that on my laptop, I crashed everything when I set it to 0 and tried the play/pause button...

<font color='red'> **IMPORTANT:** </font> In the k3d visualisation interface there is a slider named *"Objects"*. Please uncheck the "visible" box for the data sets that are not of interest. If not, as 4 volumes are generated at the same time, the visualisation will lag. Also, the interactive interface takes some time to show up (~2min) but afterwards the visualisation runs smoothly (if the structures are not all set to "visible").

In [51]:
# Creating the plot function:
def ThePlot(Sun=False, stars=False, all_data=False, duplicates=False, no_duplicate=False, line_of_sight=False, 
            trace_data=False, trace_noduplicate=False):
    plot = k3d.plot(grid_visible=False, background_color=0x000000)  # plot with no axes and a dark background

    # Adding the different data sets (i.e. with or without duplicates)
    if all_data:
        init_plot = k3d.voxels(cubes0[0], space_size=cubes0[0].shape, outlines=False, compression_level=5, 
                            color_map=[0x8B0000], name='All data')
        plot += init_plot

    if trace_data:
        plot += k3d.voxels(trace_cubes, compression_level=5, outlines=False, color_map=[0xff6666], opacity=0.1,
        name='Trace of all the data')
    
    if trace_noduplicate:
        plot += k3d.voxels(trace_cubes03, compression_level=5, outlines=False, color_map=[0xff6666], opacity=0.1,
        name='Trace of the no duplicates')

    if duplicates:
        init_plot1 = k3d.voxels(cubes01[0], compression_level=5, outlines=True, color_map=[0xff0000], 
                                name='No duplicates from SDO')
        init_plot2 = k3d.voxels(cubes02[0], compression_level=5, outlines=True, color_map=[0x0000ff],
                                name='No duplicates from STEREO')
        plot += init_plot1
        plot += init_plot2
        
    if no_duplicate:
        init_plot3 = k3d.voxels(cubes03[0], compression_level=5, outlines=True, color_map=[0x8B0000], 
                                name='No duplicates')
        plot += init_plot3
        
    if line_of_sight:
        init_plot4 = k3d.voxels(cubes1[0], compression_level=5, outlines=True, color_map=[0x0000ff], 
                                name='Seen from Stereo')
        init_plot5 = k3d.voxels(cubes2[0], compression_level=5, outlines=True, color_map=[0xff0000], 
                                name='Seen from SDO')
        plot += init_plot4
        plot += init_plot5

    if Sun:
        # Adding the SUN!!!
        plot += k3d.points(positions=points, point_size=2.5, colors=hex_colors, shader='flat', name='SUN', 
                           compression_level=5)
    if stars:
        # Adding the stars
        plot += k3d.points(positions=stars_points, point_size=50, color=0xffffff, shader='3d', name='Stars', 
                           compression_level=5)

    # Camera visualisation parameters
    plot.camera_auto_fit = False
    plot.camera_fov = 1  # FOV in degrees
    plot.camera_zoom_speed = 0.7  # it was zooming too quiclky (default=1.2)
    distance_to_sun = 100 * radius_index  # starting camera "distance" with respect to the sun's center
    plot.camera = [sun_center[0] - distance_to_sun, sun_center[1] - distance_to_sun / 2, 0,  # starting camera position
                # sun_center[0], sun_center[1], sun_center[2],  # point to look at, i.e. initial rotational reference
                cubes.shape[1]/2, cubes.shape[2]/2, cubes.shape[3]/2,
                0, 0, 1]  # up vector

    # Adding a play/pause button
    play_pause_button = widgets.ToggleButton(value=False, description='Play', icon='play')

    # Updating the voxel data
    def update_voxel(change):
        if all_data:
            init_plot.voxels = cubes0[change['new']]
        if duplicates:
            init_plot1.voxels = cubes01[change['new']]
            init_plot2.voxels = cubes02[change['new']]
        if no_duplicate:
            init_plot3.voxels = cubes03[change['new']]
        if line_of_sight:
            init_plot4.voxels = cubes1[change['new']]
            init_plot5.voxels = cubes2[change['new']]

    # Play/pause stuff
    def play():
        if play_pause_button.value and time_slider.value < len(cubes) - 1:
            time_slider.value += 1
            threading.Timer(2, play).start()  # where you also set the sleep() time. 
        else:
            play_pause_button.description = 'Play'
            play_pause_button.icon = 'play'

    def play_pause_handler(change):
        if change['new']:  # if clicked play
            play()
            play_pause_button.description = 'Pause'
            play_pause_button.icon = 'pause'
        else:  
            pass

    # Set up the time slider and the play/pause button
    time_slider = widgets.IntSlider(min=0, max=len(cubes)-1, description='Time')
    time_slider.observe(update_voxel, names='value')
    play_pause_button.observe(play_pause_handler, names='value')

    # Display
    display(plot, time_slider, play_pause_button)

In [52]:
# Plotting
ThePlot(trace_data=True, trace_noduplicate=True, Sun=True, no_duplicates=True, all_data=True)

/home/avoyeux/old_project/avoyeux/prog/python_codes/.venv/lib/python3.9/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "uint8". A coerced copy has been created.
  warnings.warn(
/home/avoyeux/old_project/avoyeux/prog/python_codes/.venv/lib/python3.9/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "bool" does not match required type "uint8". A coerced copy has been created.
  warnings.warn(
/home/avoyeux/old_project/avoyeux/prog/python_codes/.venv/lib/python3.9/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(
/home/avoyeux/old_project/avoyeux/prog/python_codes/.venv/lib/python3.9/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "int64" does not match required type "uint32". A coerced copy has been created.
  warnings.warn(


Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], camera=[-4…

IntSlider(value=0, description='Time', max=77)

ToggleButton(value=False, description='Play', icon='play')

<font color='Green'> **CONTROLS:** </font> 
- hold left click to rotate
- hold right click to move
- Scroll to zoom
- you can click on the time number and use the numpad if you don't like the slider
- there are options in the panel on the top right corner

### **Useless test**

The next part is just me trying the sparse_voxels method. I didn't end up using this method as it seems that it is less efficient than the normal k3d.voxels() method, at least for the data set used. Kept the code as it might be usefull for someone else or if the data set increases by a lot. 

In [25]:
# Sparse representation for sparse vortex
sparse_data = []
for loop, cube in enumerate(cubes):
    x_sparse, y_sparse, z_sparse = np.where(cube > 0)
    vals_sparse = cube[x_sparse, y_sparse, z_sparse]
    sparse_data.append(np.array([x_sparse, y_sparse, z_sparse, vals_sparse]).T)

KeyboardInterrupt: 

In [ ]:
# Sparse time series visualisation
sparse_used_data = sparse_data[:20]

# Creating the plot
plot = k3d.plot()

space_shape = [cubes.shape[1]+5, cubes.shape[2]+5, cubes.shape[3]+5]
init_plot = k3d.sparse_voxels(sparse_used_data[0], space_size=space_shape, compression_level=3)
plot += init_plot

# Updating the voxel data
def update_voxel(change):
    init_plot.sparse_voxels = sparse_used_data[change['new']]

# Set up the time slides
time_slider = widgets.IntSlider(min=0, max=len(used_data)-1, description='Time')
time_slider.observe(update_voxel, names='value')

# Display
display(plot, time_slider)

##### Interesting info (read only if really bored)

I also tried to use vtk, plotly and mayavi libraries. \
Plotly is super easy to use and really efficient to plot points. That being said, when I tried to plot voxels or volumes, it was super slow, borderline impossible to plot the "Rainbow" data set. \
Mayavi is even more efficient. Sadly, from what I understood and tried, it can't really plot voxels as it triangulates the surfaces around the voxels. Hence, the shapes are not all what they should be nor have the width they should have. All in all, it was really efficient and easy to use so I would guess that it is the best choice to plot really big structures where the triangulation doesn't really matter due to the number of data points. \
VTK is the go to if you are really proficient in Python (which I am not) and/or C++ as it is the library Mayavi is structured on. 